# Data preparation

In [ ]:
import pandas as pd
sv_wide_df = pd.read_pickle(open("../data/processed/20220301_straatvinken_abt_complete_df.pkl", "rb"))
sv_wide_df.head(n=3)

### Missing values

We need to adjust for missing values, which starts with knowing which columns contain them.

Luckily there's only few:

In [ ]:
#sv_wide_df.sample(n=10)[["lat", "long"]].to_csv("../data/raw/infer_coordinates.csv")

In [ ]:
sv_wide_df.isnull().sum()[sv_wide_df.isnull().sum().index[sv_wide_df.isnull().sum() > 0]]


We can neglect the img_* and lat_right, long_right, segm_distance and pano_id being NA because they won't be used. Which leaves only the number of cars / households and the verharding (road pavement type?).

For the latter we'll take the majority class (verh=1, lbl=weg met vaste verharding):

In [ ]:
print(sv_wide_df.verhlbl.value_counts())
print(sv_wide_df.verh.value_counts())

In [ ]:
sv_wide_df.verh = sv_wide_df.verh.fillna("1")

For number of households and affiliated metrics, we'll use median imputation:

In [ ]:
sv_wide_df.nhh = sv_wide_df.nhh.fillna(sv_wide_df.nhh.median())
sv_wide_df.ncars = sv_wide_df.ncars.fillna(sv_wide_df.ncars.median())
sv_wide_df.ncars_hh = sv_wide_df.ncars / sv_wide_df.nhh

Now we dealt with all missing values, we make a selection of the columns to use for training:

In [ ]:
pd.options.display.max_seq_items = 1000
display(sv_wide_df.columns)
pd.options.display.max_seq_items = 100

In [ ]:
# to be predicted columns
Y_s = ['bike', 'bus', 'car', 'truck', 'van', 'walk']
# columns to be one-hot-encoded (for categorical variables)
one_hot_cols = ['verh', 'morf', 'wegcat']
# columns that are to be left the way they are (no normalization)
num_pred_remain_cols = ['is_natw']
num_pred_segm_cols = ['segm_wall', 'segm_building',
       'segm_sky', 'segm_tree', 'segm_road', 'segm_grass', 'segm_sidewalk',
       'segm_car', 'segm_fence', 'segm_signboard', 'segm_pole', 'segm_person',
       'segm_plant', 'segm_stairs', 'segm_bridge', 'segm_streetlight',
       'segm_earth', 'segm_water', 'segm_rock', 'segm_box', 'segm_sand',
       'segm_path', 'segm_bench', 'segm_house', 'segm_van', 'segm_minibike',
       'segm_ashcan', 'segm_pot', 'segm_mountain', 'segm_door', 'segm_trade',
       'segm_field', 'segm_floor', 'segm_dirt', 'segm_flower', 'segm_truck',
       'segm_railing', 'segm_boat', 'segm_conveyer', 'segm_windowpane',
       'segm_ceiling', 'segm_stairway', 'segm_bus', 'segm_bicycle',
       'segm_chair', 'segm_cabinet', 'segm_table', 'segm_awning',
       'segm_bannister', 'segm_escalator', 'segm_bed']
num_pred_minmax_cols = ['bedi', 'pode', 'nhh', 'ncars', 'ncars_hh', 'ms_pop', 
    'acc', 'acc_death', 'acc_death30', 'acc_mort',
    'acc_ser', 'acc_sly',
    'nrijstr', 'wb', 'wegcat_H_50', 'wegcat_L2_50', 'wegcat_PII-1_50', 'wegcat_S2_50',
       'wegcat_PII-2_50', 'wegcat_S_50', 'wegcat_L1_50', 'wegcat_L_50',
       'wegcat_L3_50', 'wegcat_PI_50', 'wegcat_PII-4_50', 'wegcat_S3_50',
       'wegcat_S1_50', 'wegcat_S4_50', 'wegcat_PII_50', 'wegcat_Htot_50',
       'wegcat_Ptot_50', 'wegcat_Stot_50', 'wegcat_Ltot_50',
       'morf_130_50', 'morf_120_50', 'morf_101_50',
       'morf_102_50', 'morf_103_50', 'morf_104_50', 'morf_105_50',
       'morf_106_50', 'morf_107_50', 'morf_108_50', 'morf_109_50',
       'morf_110_50', 'morf_111_50', 'morf_112_50', 'morf_113_50',
       'morf_114_50', 'morf_116_50', 'morf_-8_50', 'morf_125_50',
       'wegcat_H_100', 'wegcat_L2_100',
       'wegcat_PII-1_100', 'wegcat_S2_100', 'wegcat_PII-2_100', 'wegcat_S_100',
       'wegcat_L1_100', 'wegcat_L_100', 'wegcat_L3_100', 'wegcat_PI_100',
       'wegcat_PII-4_100', 'wegcat_S3_100', 'wegcat_S1_100', 'wegcat_S4_100',
       'wegcat_PII_100', 'wegcat_Htot_100', 'wegcat_Ptot_100',
       'wegcat_Stot_100', 'wegcat_Ltot_100', 
       'morf_130_100', 'morf_120_100', 'morf_101_100', 'morf_102_100',
       'morf_103_100', 'morf_104_100', 'morf_105_100', 'morf_106_100',
       'morf_107_100', 'morf_108_100', 'morf_109_100', 'morf_110_100',
       'morf_111_100', 'morf_112_100', 'morf_113_100', 'morf_114_100',
       'morf_116_100', 'morf_-8_100', 'morf_125_100', 
       'wegcat_H_150', 'wegcat_L2_150', 'wegcat_PII-1_150', 'wegcat_S2_150',
       'wegcat_PII-2_150', 'wegcat_S_150', 'wegcat_L1_150', 'wegcat_L_150',
       'wegcat_L3_150', 'wegcat_PI_150', 'wegcat_PII-4_150', 'wegcat_S3_150',
       'wegcat_S1_150', 'wegcat_S4_150', 'wegcat_PII_150', 'wegcat_Htot_150',
       'wegcat_Ptot_150', 'wegcat_Stot_150', 'wegcat_Ltot_150',
       'morf_130_150', 'morf_120_150',
       'morf_101_150', 'morf_102_150', 'morf_103_150', 'morf_104_150',
       'morf_105_150', 'morf_106_150', 'morf_107_150', 'morf_108_150',
       'morf_109_150', 'morf_110_150', 'morf_111_150', 'morf_112_150',
       'morf_113_150', 'morf_114_150', 'morf_116_150', 'morf_-8_150',
       'morf_125_150', 'wegcat_H_250',
       'wegcat_L2_250', 'wegcat_PII-1_250', 'wegcat_S2_250',
       'wegcat_PII-2_250', 'wegcat_S_250', 'wegcat_L1_250', 'wegcat_L_250',
       'wegcat_L3_250', 'wegcat_PI_250', 'wegcat_PII-4_250', 'wegcat_S3_250',
       'wegcat_S1_250', 'wegcat_S4_250', 'wegcat_PII_250', 'wegcat_Htot_250',
       'wegcat_Ptot_250', 'wegcat_Stot_250', 'wegcat_Ltot_250',
       'morf_130_250', 'morf_120_250',
       'morf_101_250', 'morf_102_250', 'morf_103_250', 'morf_104_250',
       'morf_105_250', 'morf_106_250', 'morf_107_250', 'morf_108_250',
       'morf_109_250', 'morf_110_250', 'morf_111_250', 'morf_112_250',
       'morf_113_250', 'morf_114_250', 'morf_116_250', 'morf_-8_250',
       'morf_125_250', 'wegcat_H_500',
       'wegcat_L2_500', 'wegcat_PII-1_500', 'wegcat_S2_500',
       'wegcat_PII-2_500', 'wegcat_S_500', 'wegcat_L1_500', 'wegcat_L_500',
       'wegcat_L3_500', 'wegcat_PI_500', 'wegcat_PII-4_500', 'wegcat_S3_500',
       'wegcat_S1_500', 'wegcat_S4_500', 'wegcat_PII_500', 'wegcat_Htot_500',
       'wegcat_Ptot_500', 'wegcat_Stot_500', 'wegcat_Ltot_500',
       'morf_130_500', 'morf_120_500',
       'morf_101_500', 'morf_102_500', 'morf_103_500', 'morf_104_500',
       'morf_105_500', 'morf_106_500', 'morf_107_500', 'morf_108_500',
       'morf_109_500', 'morf_110_500', 'morf_111_500', 'morf_112_500',
       'morf_113_500', 'morf_114_500', 'morf_116_500', 'morf_-8_500',
       'morf_125_500', 'wegcat_H_1000',
       'wegcat_L2_1000', 'wegcat_PII-1_1000', 'wegcat_S2_1000',
       'wegcat_PII-2_1000', 'wegcat_S_1000', 'wegcat_L1_1000', 'wegcat_L_1000',
       'wegcat_L3_1000', 'wegcat_PI_1000', 'wegcat_PII-4_1000',
       'wegcat_S3_1000', 'wegcat_S1_1000', 'wegcat_S4_1000', 'wegcat_PII_1000',
       'wegcat_Htot_1000', 'wegcat_Ptot_1000', 'wegcat_Stot_1000',
       'wegcat_Ltot_1000', 'morf_130_1000',
       'morf_120_1000', 'morf_101_1000', 'morf_102_1000', 'morf_103_1000',
       'morf_104_1000', 'morf_105_1000', 'morf_106_1000', 'morf_107_1000',
       'morf_108_1000', 'morf_109_1000', 'morf_110_1000', 'morf_111_1000',
       'morf_112_1000', 'morf_113_1000', 'morf_114_1000', 'morf_116_1000',
       'morf_-8_1000', 'morf_125_1000', 
       'wegcat_H_2000', 'wegcat_L2_2000', 'wegcat_PII-1_2000',
       'wegcat_S2_2000', 'wegcat_PII-2_2000', 'wegcat_S_2000',
       'wegcat_L1_2000', 'wegcat_L_2000', 'wegcat_L3_2000', 'wegcat_PI_2000',
       'wegcat_PII-4_2000', 'wegcat_S3_2000', 'wegcat_S1_2000',
       'wegcat_S4_2000', 'wegcat_PII_2000', 'wegcat_Htot_2000',
       'wegcat_Ptot_2000', 'wegcat_Stot_2000', 'wegcat_Ltot_2000',
       'morf_130_2000', 'morf_120_2000',
       'morf_101_2000', 'morf_102_2000', 'morf_103_2000', 'morf_104_2000',
       'morf_105_2000', 'morf_106_2000', 'morf_107_2000', 'morf_108_2000',
       'morf_109_2000', 'morf_110_2000', 'morf_111_2000', 'morf_112_2000',
       'morf_113_2000', 'morf_114_2000', 'morf_116_2000', 'morf_-8_2000',
       'morf_125_2000']
print(f"Y_s {len(Y_s)}")
print(f"one_hot_cols {len(one_hot_cols)}")
print(f"num_pred_remain_cols {len(num_pred_remain_cols)}")
print(f"num_pred_segm_cols {len(num_pred_segm_cols)}")
print(f"num_pred_minmax_cols {len(num_pred_minmax_cols)}")
print(f"sum pred cols {len(one_hot_cols) + len(num_pred_remain_cols) + len(num_pred_segm_cols) + + len(num_pred_minmax_cols)}")
pred_cols = one_hot_cols.copy()
pred_cols.extend(num_pred_remain_cols)
pred_cols.extend(num_pred_segm_cols)
pred_cols.extend(num_pred_minmax_cols)
print("actual # pred cols", len(pred_cols))


Exporting columns to YAML file

In [ ]:
import yaml
COLUMN_CONFIG_FILE = "../src/aicityflowsstraatvinken/columns_2022_1.3.yaml"
config= {
    "iteration": "2022 1.3",
    "date_created": date.today().isoformat(),
    "Y_s": Y_s, 
    "one_hot_cols": one_hot_cols, 
    "num_pred_remain_cols": num_pred_remain_cols,
    "num_pred_minmax_cols": num_pred_minmax_cols,
    "num_pred_segm_cols": num_pred_segm_cols
}
with open(COLUMN_CONFIG_FILE, 'w') as f:
    config = yaml.dump(config, stream=f,
                       default_flow_style=False, sort_keys=False)

With the columns selected, we can make a split of the dataset in a train and test set:.

Some important considerations:
* when the dataset consists of the whole of Flanders (at time of writing this is not the case), being able to pick evenly from all provinces could be interesting

In [ ]:
import time
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline #, make_union
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.linear_model import ElasticNetCV
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from catboost import CatBoostRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
import pickle
from datetime import date
import numpy as np

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    sv_wide_df[pred_cols], 
    sv_wide_df[Y_s], 
    test_size=.2, 
    random_state=42,
    shuffle=True,
    #stratify=sf_wide_df[""]
)

pickle.dump((X_train, X_test, y_train, y_test), open(f"../data/training/{date.today().isoformat().replace('-', '')}_train_test_data.pkl", 'wb'))
       

# Baseline performance

For the sake of having a frame of reference to which to compare our model's accuracy, we can calculate some baseline performance metrics:
* the mean value
* the median value

In [ ]:
n_samples = y_test.shape[0]
maea = mean_absolute_error(y_test['walk'], [y_train["walk"].mean()] * n_samples)
rmsea = np.sqrt(mean_squared_error(y_test['walk'], [y_train["walk"].mean()]*y_test.shape[0]))
print(f"Average value MAE {maea:8.4f} RMSE {rmsea:8.4f}")
maem = mean_absolute_error(y_test['walk'], [y_train["walk"].median()]*n_samples)
rmsem = np.sqrt(mean_squared_error(y_test['walk'], [y_train["walk"].median()]*y_test.shape[0]))
print(f"Median value  MAE {maem:8.4f} RMSE {rmsem:8.4f}")
     

We can repeat this exercise for any percentile and plot the lowest error we could achieve this way:

In [ ]:

q = np.linspace(0, 1, 101)
mae = [mean_absolute_error(y_test["walk"], [quant]*n_samples) for quant in np.quantile(y_train['walk'], q)]
rmse = [np.sqrt(mean_squared_error(y_test["walk"], [quant]*n_samples)) for quant in np.quantile(y_train['walk'], q)]
quantdf = pd.DataFrame({"quantile": q, "mae": mae, "rmse": rmse})

fig, ax=plt.subplots(figsize=(10, 5))
ax.scatter(quantdf["quantile"], quantdf.mae, label="mae")
ax.scatter(quantdf["quantile"], quantdf.rmse, label="rmse")
ax.set_xlabel("quantile of training data")
ax.set_ylabel("error")
ax.set_ylim(0, 100)
plt.legend()
plt.show()
print(f"min mae  {quantdf.mae.min():8.4f} \nmin rmse {quantdf.rmse.min():8.4f}")


In [ ]:

quantdf[quantdf.mae == quantdf.mae.min()]
quantdf[quantdf.rmse == quantdf.rmse.min()]

# Model training

In [ ]:
#!/opt/anaconda3/envs/straatvinken/bin/pip install pytorch-tabnet
#!/opt/anaconda3/envs/straatvinken/bin/pip install lightgbm

In [ ]:
class Columns(BaseEstimator, TransformerMixin):
    def __init__(self, names=None):
        self.names = names

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X):
        return X[self.names]

In [ ]:
one_hot_categories = list(sv_wide_df[one_hot_cols].apply(lambda x: list(set(x)), axis=0).values)
prefix = np.hstack([[one_hot_cols[ix]] * (len(cat_list)-1) for ix, cat_list in enumerate(one_hot_categories)]) 
suffix =  np.hstack([cat_list[1:] for cat_list in one_hot_categories]).astype(str)
one_hot_col_names = [f"{prefix}_{suffix}" for prefix, suffix in zip(prefix, suffix)]
one_hot_col_names

## Exploration of the training data


In [ ]:
num_pred_minmax = num_pred_minmax_cols.copy()
num_pred_minmax.extend(num_pred_segm_cols)
_pipeline_def = ("features", FeatureUnion([
    ('ohe', make_pipeline(
        Columns(names=one_hot_cols),
        OneHotEncoder(sparse=False, drop="first", categories=one_hot_categories, handle_unknown="error"))),
    ('mima', make_pipeline(
        Columns(names=num_pred_minmax),
        MinMaxScaler())),
    ('keep', make_pipeline(Columns(names=num_pred_remain_cols)))
]))
data_transformation_pipe = Pipeline(
    [_pipeline_def]
)

_X_trans = data_transformation_pipe.fit_transform(X_train)

cols = one_hot_col_names.copy()
cols.extend(num_pred_minmax)
cols.extend(num_pred_remain_cols)
_X_train_df = pd.DataFrame(_X_trans, columns=cols)
_X_train_df

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, fixed
import matplotlib.pyplot as plt
import seaborn as sns

sel=widgets.SelectMultiple(
    options=cols, 
    value=cols[:10], 
    description="columns",
    #rows=5,
)

def show_corr(corr_data, rows):
    general_cols=["bike", "walk", "bus","car", "truck", "van"]
    wegcat_cols = rows

    fig, ax = plt.subplots(figsize=(15, 15))
    cmap = sns.color_palette("Spectral", as_cmap=True)
    sns.heatmap(corr_data.loc[rows, general_cols], annot=True, cmap=cmap, center=0, vmin=-1, vmax=1)
    ax.xaxis.tick_top()
    plt.title(f"Correlation plot")
    plt.show()

@interact(cols=sel, plottype=["corr", "hist"])
def show_plots(cols, plottype):
    frag_df = pd.concat([_X_train_df[list(cols)], y_train], axis=1)
    if plottype=="hist":
        fig, ax = plt.subplots(figsize=(12, 15))
        frag_df[list(cols)].hist(ax=ax)
        plt.show()
    else:
        corr = frag_df.corr()
        show_corr(corr, rows=cols)
    

### Training the Elastic Net using Cross Validation

In [ ]:
#!/opt/anaconda3/envs/straatvinken/bin/pip install --upgrade scikit-learn

In [ ]:
from sklearn import set_config
set_config(display="diagram")

In [ ]:
pipe = Pipeline(
    [
        _pipeline_def,
        ('est', ElasticNetCV(cv=10, random_state=42))
    ]
)




#_gs = GridSearchCV(pipe, param_grid=[params], scoring='neg_mean_absolute_error', cv=10)
result = pipe.fit(X_train, y_train['walk']) 
result

# Elastic Net fit results

Let's analyse the results of the model fit. 
The code below shows
* selected hyperparameter values (alpha and l1_ratio)
* beta coefficients (some of which are zeroes due to l1 penalty shrinkage)
* the intercept

In [ ]:

np.corrcoef()

In [ ]:
print(f"alpha {pipe.named_steps['est'].alpha_:.4f}: The amount of penalization choosen by cross validation")

print(f"l1_ratio {pipe.named_steps['est'].l1_ratio_:.4f}: The compromise between l1 and l2 penalization choosen by cross validation")

coeffs = {col: coeff for col, coeff in zip(cols, pipe.named_steps['est'].coef_)}

ZERO_THRESHOLD = 0.0001
zeroes = len([val for val in coeffs.values() if abs(val) < ZERO_THRESHOLD])
coeffs = dict(sorted(coeffs.items(), key=lambda item: abs(item[1]), reverse=True))

print(f"coefficients: parameter vector (zeroes: {zeroes}/{len(cols)})")
MAX_ITEMS=40
[print(f'   {col:<16}: {coeff:8.4f}') for ix, (col, coeff) in enumerate(coeffs.items()) if abs(coeff) > ZERO_THRESHOLD and ix < MAX_ITEMS]
print("    ...")
print(f"intercept {pipe.named_steps['est'].intercept_:.4f}")
#print(f"mse_path {pipe.named_steps['est'].mse_path_}")


As we can see, the 10 largest (absolute) coefficients are assigned to 
   * ms_pop (pop density in square grid): 43.8277
   * segm_trade (segmentation data: %trade name, brand name): 32.0203
   * ncars_hh (# cars per household in statistical sector): -31.1926
   * bedi (building density): 23.3342
   * segm_sky (segmentation data: % sky): -23.2176
   * ncars (# cars in statistical sector): -20.2385
   * morf_110_50 (# roads of morphology type 110 aka "ventweg" within 50m): 18.6296
   * segm_person (segmentation data: % persons): 16.5171
   * segm_awning (segmentation data: % awning, sunshade, sunblind): 16.4896
   * nhh (# households in statistical sector): 15.5749
   
Let's see how well the model did, in terms of MAE and RMSE:

### Error analysis

Let's plot the mean absolute error in a histogram:

In [ ]:
(result.predict(X_train) - y_train['walk']).hist(bins=25)

plt.title("Prediction errors on training data")
plt.show()

In [ ]:
(result.predict(X_test) - y_test['walk']).hist(bins=25)
plt.title("Prediction errors on test data")
plt.show()

Let's for a minute look at the predictions for which the error was very large (> 150)

In [ ]:
X_test["ElasticNet_abs_error_walk"] = result.predict(X_test) - y_test['walk']
X_test["ElasticNet_large_error_walk"] = abs(X_test["ElasticNet_abs_error_walk"]) >= 150

In [ ]:
X_test[X_test["ElasticNet_large_error_walk"]].sort_values(by="ElasticNet_abs_error_walk", ascending=True)

# Catboost regressor

In [ ]:
pipe_cb = Pipeline(
    [
        _pipeline_def
    ]
)

pipe_cb.fit_transform(X_train)


In [ ]:

model = CatBoostRegressor()

grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

grid_search_result = model.grid_search(grid,
                                       X=pipe_cb.fit_transform(X_train),
                                       y=y_train["walk"],
                                       plot=True)

In [ ]:
grid_search_result

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
print("mean abs error", mean_absolute_error(model.predict(pipe_cb.fit_transform(X_test)), y_test["walk"]))
print("mean squared error", np.sqrt(mean_squared_error(model.predict(pipe_cb.fit_transform(X_test)), y_test["walk"])))


In [ ]:
quentin_gpd

In [ ]:
import geopandas as gpd
import pandas as pd

quentin_df = pd.read_csv("../data/raw/infer_coordinates_quentin_large.csv")
quentin_df
quentin_gpd = gpd.GeoDataFrame(quentin_df, geometry=gpd.geoseries.from_wkt(quentin_df.representative_point.values, crs=4326))
quentin_df
display(quentin_gpd)
quentin_gpd.shape

In [12]:
quentin_df[["long", "lat"]] = quentin_df.apply(lambda p: (p.geometry.x, p.geometry.y), axis=1, result_type="expand")


In [14]:
#quentin_df = quentin_df.drop(columns=["lon"])

In [15]:
quentin_df.to_csv("../data/raw/infer_coordinates_quentin_large.csv", index=False)

In [ ]:
import pickle
df = pickle.load(open("../data/processed/20220303-infer_output.pkl", "rb"))
df

In [ ]:
quentin_df.shape

In [ ]:
quentin_df = quentin_df.loc[~quentin_df.index.duplicated(keep='first')]